# Assignment 1 Sistem Temu Kembali Informasi
<b> Lintang Birda Saputra (22/505034/PA/21737)</b> 
<br>
<b> Kelas KOM </b>
<br>
<b> Materi Inverted Index dan Boolean Query </b>

In [12]:
import pandas as pd
import nltk
import os
import sys
import pickle as pkl
import array
import contextlib

# nltk.download('popular')


data = pd.read_csv("./News.csv")


Inverted Index

1. Explore the dataset that contains news from several authors, provide the statistical details of this document collections (how many docs, words/terms, sentences). Download the dataset from this link https://drive.google.com/file/d/13UUj240WKjosQnLfnveMcOutd0OYxBDF/view?usp=sharing  and use text under the "content" column.

2. Create inverted index based on the document collections (term based)

3. Create simple Boolean Retrieval, for merging (AND), union (OR) and negation (NOT). Check the performance of your Boolean Retrieval, by

trying at least 5 queries for each of the logical operator.

4. Record the time needed for creating inverted index and for retrieving queries.

5. Apply query optimization (if necessary) and then compare the again the time needed for retrieving queries.





Things to be submitted:

1. A 2 pages of report, explaining the analysis of the results

2. Link to the code (I prefer if you use Google Colab and provide the accessible link)

3. To write the code, please use Python.

In [13]:
data.head()

,id,id_author,title,portal,time,author,editor,content,source
0,0,1,Infografis Pekerja Asing Dilarang Masuk Wilaya...,Liputan6.com,"24 Jul 2021, 09:02 WIB",Abdillah,Abdillah,Pemerintah melalui Menteri Hukum dan Hak Asasi...,https://www.liputan6.com/news/read/4614451/inf...
1,1,1,Infografis Jadwal Bulu Tangkis Indonesia di Ol...,Liputan6.com,"23 Jul 2021, 23:23 WIB",Abdillah,Abdillah,Bulu Tangkis menjadi andalan Indonesia berburu...,https://www.liputan6.com/bola/read/4614427/inf...
2,2,1,"Infografis Jangan Bebal, Kamu Tidak Kebal Covi...",Liputan6.com,"23 Jul 2021, 10:40 WIB",Abdillah,Abdillah,Covid-19 tidak mengenal usia dan status. Siapa...,https://www.liputan6.com/news/read/4613233/inf...
3,3,1,Infografis Awas Perokok Lebih Rentan Tertular ...,Liputan6.com,"22 Jul 2021, 10:35 WIB",Abdillah,Abdillah,Kebiasaan merokok berisiko menimbulkan sejumla...,https://www.liputan6.com/news/read/4612324/inf...
4,4,1,Infografis Perbedaan Aturan PPKM Level 3 dan 4,Liputan6.com,"22 Jul 2021, 09:01 WIB",Abdillah,Abdillah,Pemberlakuan Pembatasan Kegiatan Masyarakat at...,https://www.liputan6.com/news/read/4612511/inf...


### Statistical Details of Document Collections

In [14]:
data = data.dropna(subset=['content'])
lenDocs  = len(data['content'])
print("jumlah dokumen: ",lenDocs)

totalSentences = 0
totalWords = 0
doc_total_sentences = []
doc_total_words = []
for doc in data['content']:
  totalSentences += len(nltk.sent_tokenize(doc))
  doc_total_sentences.append(len(nltk.sent_tokenize(doc)))
  totalWords += len(nltk.word_tokenize(doc))
  doc_total_words.append(len(nltk.word_tokenize(doc)))

print("total kalimat di seluruh dokumen: ", totalSentences)
print("total kata di seluruh dokumen: ", totalWords)
data= data.assign(total_sentences = doc_total_sentences)
data = data.assign(total_words = doc_total_words)

data.head()

jumlah dokumen:  14334
total kalimat di seluruh dokumen:  352491
total kata di seluruh dokumen:  6510359


,id,id_author,title,portal,time,author,editor,content,source,total_sentences,total_words
0,0,1,Infografis Pekerja Asing Dilarang Masuk Wilaya...,Liputan6.com,"24 Jul 2021, 09:02 WIB",Abdillah,Abdillah,Pemerintah melalui Menteri Hukum dan Hak Asasi...,https://www.liputan6.com/news/read/4614451/inf...,8,115
1,1,1,Infografis Jadwal Bulu Tangkis Indonesia di Ol...,Liputan6.com,"23 Jul 2021, 23:23 WIB",Abdillah,Abdillah,Bulu Tangkis menjadi andalan Indonesia berburu...,https://www.liputan6.com/bola/read/4614427/inf...,8,107
2,2,1,"Infografis Jangan Bebal, Kamu Tidak Kebal Covi...",Liputan6.com,"23 Jul 2021, 10:40 WIB",Abdillah,Abdillah,Covid-19 tidak mengenal usia dan status. Siapa...,https://www.liputan6.com/news/read/4613233/inf...,12,124
3,3,1,Infografis Awas Perokok Lebih Rentan Tertular ...,Liputan6.com,"22 Jul 2021, 10:35 WIB",Abdillah,Abdillah,Kebiasaan merokok berisiko menimbulkan sejumla...,https://www.liputan6.com/news/read/4612324/inf...,10,114
4,4,1,Infografis Perbedaan Aturan PPKM Level 3 dan 4,Liputan6.com,"22 Jul 2021, 09:01 WIB",Abdillah,Abdillah,Pemberlakuan Pembatasan Kegiatan Masyarakat at...,https://www.liputan6.com/news/read/4612511/inf...,9,114


In [15]:
class IdMap:
    """maping dari string ke id dan sebaliknya """
    def __init__(self):
        self.str_to_id = {}
        self.id_to_str = []

    def __len__(self):
        return len(self.id_to_str)

    def _get_str(self, i):
        return self.id_to_str[i]

    def _get_id(self, s):
        if s not in self.str_to_id:
            self.str_to_id[s] = len(self.id_to_str)
            self.id_to_str.append(s)
        return self.str_to_id.get(s)

    def __getitem__(self, key):
        if type(key) is int:
            return self._get_str(key)
        elif type(key) is str:
            return self._get_id(key)
        else:
            raise TypeError

In [16]:
def encode_postings_list(posting_lists):
    """encode posting lists ke stream of bytes agar bisa disimpan di file
    
    Params
    ------ 
    posting_lists: List[int]
        list of docIDs

    Returns
    ------
    bytes
        bytearray berisi integer docIds di dalam posting_list
    """
    return array.array('L', posting_lists).tobytes()

def decode_postings_list(encoded_postings_list):
    """decode posting lists dari stream of bytes
    

    Params
    ------
    encoded_postings_list: bytes
        bytearray berisi posting lists hasil dari encoding

    Returns
    ------
    List[int]
        posting_lists yang sudah didecode/list of docIDs
    """
    decoded_postings_list = array.array('L')
    decoded_postings_list.frombytes(encoded_postings_list)
    return decoded_postings_list.tolist()

In [17]:
class InvertedIndex:
    """
    Class ini menimplemntasikan method untuk read dan write inverted index ke disk

    Attr
    -------
    postings_dict = dictionary[term](start_position_in_index_file, 
                                                number_of_postings_in_list,
                                               length_in_bytes_of_postings_list)
        memetakan termID ke metadata dari posting lists tersebut yang digunakan 
        untuk reading dan writing postings ke disk. start_position_in_index_file adalah posisi awal
        dari posting lists di dalam index file (digunakan saat file.seek(start_position_in_index_file)).
        number_of_postings_in_list adalah jumlah docIDs didalam postings.
        length_in_bytes_of_postings_list adalah ukuran postings di dalam index file (digunakan untuk 
        read posting dari index file -> file.read(length_in_bytes_of_postings_list) dari posisi start_position_in_index_file
        -> file.seek(start_position_in_index_file)).
        
        
    terms: List[int]
        list of termID
    """
    def __init__(self, index_name,  directory=''):
      """
      Params
      ------
      index_name: string
          nama index file
      postings_encoding: class
          class untuk encoding/decoding
      directory: 
          directory untuk menyimpan index files
          
      """
      self.index_file_path = os.path.join(directory, index_name+'.index')
      self.metadata_file_path = os.path.join(directory, index_name+'.dict')

   
      self.directory = directory

      self.postings_dict = {}
      self.terms = []

    def open_writer(self):
        self.index_file = open(self.index_file_path, 'wb+')
        return self
        
    def __enter__(self):
        """Membuka file inverted index dan file metadata , serta load metadata  postings_dict, terms"""
        self.index_file = open(self.index_file_path, 'rb+')

        # load posting list dan terms dari file metadata
        with open(self.metadata_file_path, 'rb') as f:
            self.postings_dict, self.terms = pkl.load(f)
            self.term_iter = self.terms.__iter__()

        return self
    

    def __exit__(self, exception_type, exception_value, traceback):
        """"close file inverted index  dan save file inverted index"""

        self.index_file.close()

        with open(self.metadata_file_path, 'wb') as f:
            pkl.dump([self.postings_dict, self.terms], f)
            
    def exit(self):
        self.index_file.close()

        with open(self.metadata_file_path, 'wb') as f:
            pkl.dump([self.postings_dict, self.terms], f)
        

    def __getitem__(self, key):
        return self._get_postings_list(key)

    def _get_postings_list(self, term):
        """"
        mendapatkan postinglists dari term

        Params
        -----
        Int
            termID 
        """
        try:
            start_position, n_postings, length_in_bytes = self.postings_dict[term]
            self.index_file.seek(start_position)
            return decode_postings_list(self.index_file.read(length_in_bytes))
        except KeyError:
            return []

    def append(self, term, postings_list):
        """
        append term dan posting lists ke index file (posisi di block paling akhir di file)
    
        posting_dict memetakan termID ke tuple, isi posting_dict:
        (start_position_in_index_file,
               number_of_postings_in_list,
               length_in_bytes_of_postings_list)
        """
        encoded_postings_list = encode_postings_list(postings_list) # bytearrray dari posting_lists
        start_position_in_index_file = self.index_file.seek(0, 2) # posisi paling akhir di file
        length_in_bytes_of_postings_list = self.index_file.write(encoded_postings_list) # write bytearray posting_lists ke file index
        self.terms.append(term) # tambahkan term ke list of terms
        self.postings_dict[term] = (start_position_in_index_file, len(postings_list),length_in_bytes_of_postings_list ) # simpan metadata dari term



In [18]:
class InvertedIndexIterator(InvertedIndex):
  """
        iterator untuk setiap block inverted index.
        read file dari disk hanya dilakukan saat process merge (ketika heapq.merge(*indices) 
            method __next__ dari iterator akan dipanggil dan read posting lists dari term ke disk).
       setiap iterasi, yang diload ke memory hanya satu posting list dari disk. tujuannya agar posting lists yang 
       di load ke memory tidak terlalu besar/tidak semua posting lists yang di 
        disk di load ke memory. 
  """
  def enter(self):
    super().__enter__()
    self.curr_term_pos = 0 
    return self 


  def __iter__(self):
   
    return self

  def __next__(self):
    """
        return  pasangan (termID, posting_lists) selanjutnya
        

        read satu posting lists dari term dari index file/disk

        Returns
        -----
        Int
            termID yang di read
        List[Int]
            posting lists dari termID
        
    """
    if self.curr_term_pos < len(self.terms):
        term = self.terms[self.curr_term_pos]
        self.curr_term_pos += 1
        start_position, n_postings, length_in_bytes = self.postings_dict[term]
        self.index_file.seek(start_position)
        postings_list = decode_postings_list(self.index_file.read(length_in_bytes))
        return term, postings_list
    else:
        raise StopIteration


  def exit(self):
      """
        close file index
      """
      super().exit()       

### Inverted Index 


In [19]:
import numpy as np
import time 
import heapq
from collections import defaultdict
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory


factory = StemmerFactory()
stemmer = factory.create_stemmer()


stop_words = set(nltk.corpus.stopwords.words('indonesian'))


class SIPMI_BSBIIndexer:
  """Blocked sort-based Indexing Class
  1. mensegmentasi seluruh dokumen di dalam csv menjadi 11 block/bagian.
  2. sorting pasangan termID-pairID setiap block berdasarkan termID lalu sort berdasarkan pairID dan buat posting lists .
  3. menyimpan sorted posting lists ke index file di disk.
  4. merge semua block inverted index yang ada di intermediate_indices dan write ke disk (hasil akhir inverted index).

  Attrs
  ------
  term_id_map: IdMap
      untuk mapping term dari string ke termID
  doc_id_map: IdMap
    untuk mapping doc dari title string ke docID
  file_path: str
      path ke file data input csv yang akan di index
  output_dir: str
      path directory output untuk menyimpan setiap block inverted index dan merged index
  index_name: str
      nama merged index (file index berisi semua posting lists dari data input)
  postings_encoding: 
      encoding untuk menyimpan postings list ke disk (default nya UncompressedPostings).
  docWordCount: dict[Int][Int]
      menyimpan jumlah kata di dalam dokumen
    
  """
  def __init__(self, file_path, output_dir, index_name = "SIPMI_BSBI_lintang",
                 ):
    self.term_id_map = IdMap()
    self.doc_id_map = IdMap()
    self.file_path = file_path
    self.output_dir = output_dir
    self.index_name = index_name
    self.docLen = lenDocs
    self.intermediate_indices = []
    self.docWordCount =  {}


  def save(self):
      """menyimpan doc_id_map dan term_id_map ke output directory"""

      with open(os.path.join(self.output_dir, 'terms.dict'), 'wb') as f:
          pkl.dump(self.term_id_map, f)
      with open(os.path.join(self.output_dir, 'docs.dict'), 'wb') as f:
          pkl.dump(self.doc_id_map, f)

  def load(self):
      """Load doc_id_map and term_id_map dari output directory"""

      with open(os.path.join(self.output_dir, 'terms.dict'), 'rb') as f:
          self.term_id_map = pkl.load(f)
      with open(os.path.join(self.output_dir, 'docs.dict'), 'rb') as f:
          self.doc_id_map = pkl.load(f)

  def inverted_index_from_doc(self, block, i):
     # membuat inverted index untuk setiap block 
     td_pairs = self.parse_block(block)
     index_id = 'index_'+str(i) # membuat inverted index setiap block
      

     index = InvertedIndex(index_id, directory=self.output_dir).open_writer()
        
        # membuka file index_id dan write posting list ke dalam file tersebut
        # setelah itu ke block berikutnya dan close file index_id
     self.invert_write(td_pairs, index)
     td_pairs = None

     self.intermediate_indices.append(index_id)
     index.exit()
     print("berhasil memproses index ke-", i)

  def index(self):
      """
        indexing semua document di file csv menggunakan Block sorted-based indexing
      """
      
      start = time.time()
      
      chunksize =  1434
      # data = pd.read_csv(self.file_path)
      file_name = self.file_path.split("/")[-1]
      i = 0
      # read chunk of csv dengan setiap chunk berukuran 1434
      for block in pd.read_csv(self.file_path, chunksize=chunksize):
         # membuat inverted index untuk setiap block
        self.inverted_index_from_doc(block, i)
        i +=1   
    

      self.save()
      merged_index  = InvertedIndex(self.index_name, directory=self.output_dir ).open_writer() 
          # membuka file index_name dan merge semua block inverted index ke merged_index
          # write merged_index ke file index_name
      indices = []
      try:
          for index_id in self.intermediate_indices:
              
                curr_idx = InvertedIndexIterator(index_id,
                                        directory=self.output_dir).enter()
                indices.append(curr_idx)
          self.merge(indices, merged_index)
      except:
          print(f"An error occurred: {e}")
      finally:
        for indices in indices:
            indices.exit()      
        
      merged_index.exit()

      end = time.time()
      print(f"total waktu yang dibutuhkan untuk indexing seluruh dokumen: {(end-start)*10**3:.03f}ms")


  def merge(self, indices, merged_index):
        """
        merge semua blok inverted index menjadi satu index (merged_index)
        dan write merged_index ke file index (melalui method append() dari InvertedIndex

        Params
        ------
        indices: List[InvertedIndexIterator]
            list of InvertedIndexIterator, setiap elemen nya adalah satu block inverted index (hasil
            dari split dokumen menjadi 11 block inverted index).
        merged_index: InvertedIndex
            hasil akhir inverted index, dimana setiap block inverted index dimerge ke merged_index ini.
        """
        last_term = last_posting = None
        for curr_term, curr_postings in heapq.merge(*indices):
            # heapq.merge mereturn list of curr_term dan curr_postings yang sudah disort berdasarkan term dan postings_list
            # posting lists didapatkan dari read block of bytes ke index file yang ada didalam disk (method __next__ InvertedIndexIterator)
            # 
            if curr_term != last_term:
                if last_term:
                    last_posting = list(sorted(last_posting)) # sort previous posting lists berdasarkan docIDs
                    merged_index.append(last_term, last_posting) # write previous merged posting lists ke file index
                last_term = curr_term
                last_posting = curr_postings
            else:
                last_posting += curr_postings
        if last_term:
            last_posting = list(sorted(last_posting)) 
            merged_index.append(last_term, last_posting) # write merged posting lists ke file index


  def parse_block(self, block):
      """
        Parse satu block yang berisi list of documents dari csv menjadi termID-docID pairs
    
        Params
        ------
        block: pandas.Dataframe
           satu block dataframe (hasil dari read csv dengan ukuran chunksize)

        Returns
        ------
        List[tuple[Int, Int]]
            list semua termID, docID pairs
        
      """
      term_doc_pairs = []

      for ind in block.index:
        
        doc = block["content"][ind]
        if type(doc) != str:
          continue
        processed_word_in_doc = []
        doc_id = self.doc_id_map[block["title"][ind]]
        doc = stemmer.stem(doc) # stemming
        words = nltk.word_tokenize(doc) # tokenisasi
        self.docWordCount[doc_id] = len(words) # simpan word count dokumen untuk hitung tf
          
        for word in words:
          word = word.lower()  # mengubah kata ke lowercase
          if word not in stop_words: # stopword removal
              term_id = self.term_id_map[word]
              term_doc_pairs.append([term_id, doc_id])
      return term_doc_pairs

  def invert_write(self, td_pairs, index):
    """
    Invert td_pairs menjadi posting lists .
     sorting berdasarkan termID lalu sort lagi berdasarkan docID.
    lalu simpan posting list ke file indeex melalui method append() dari InvertedIndex
    
     Params
     ------
     td_pairs: List[Tuple[Int,Int]]
         list semua termID, docID pairs
     index: InvertedIndex
         satu block inverted index 
    """
    term_doc_dict =  defaultdict(list)
    for t,d in td_pairs:
        term_doc_dict[t].append(d)
    for t in sorted(term_doc_dict.keys()):
      unsorted_posting_list = term_doc_dict[t]

      sorted_posting_list = sorted(unsorted_posting_list)
    
      index.append(t, sorted(sorted_posting_list))


In [20]:
def sorted_intersect(list1, list2):
    """

    operasi intersection dari list1 dan list2.
    list1 dan list2 sorted by docID.

    Params
    ----
    list1: List[Int]
        posting lists ke-1
    list2: List[Int]
        posting lists ke-2

    Returns
    -----
    List[Int]
        hasil dari operasi intersection
    """
    idx1 = idx2 = 0
    intersect = []
    while idx1 < len(list1) and idx2 < len(list2):
        if list1[idx1] < list2[idx2]:
            idx1 += 1
        elif list2[idx2] < list1[idx1]:
            idx2 += 1
        else:
            intersect.append(list1[idx1])
            idx1 += 1
            idx2 += 1
    return intersect

def sorted_union(list1, list2):
    """

    operasi union dari list1 dengan list2.
    list1 dan list2 sorted by docID.

    Params
    ----
    list1: List[Int]
        posting lists ke-1
    list2: List[Int]
        posting lists ke-2

    Returns
    -----
    List[Int]
        hasil dari operasi union
    """
    idx1 = idx2 = 0
    union = []
    while idx1 < len(list1) and idx2 < len(list2):
        if list1[idx1] < list2[idx2]:
            union.append(list1[idx1])
            idx1 += 1
        elif list1[idx1] > list2[idx2]:
            union.append(list2[idx2])
            idx2 += 1
        else:
            union.append(list1[idx1])
            idx1 +=1
            idx2 += 1
    if idx1 == len(list1):
        while(idx2 < len(list2)):
            union.append(list2[idx2])
            idx2 += 1
    else:
        while(idx1 < len(list1)):
            union.append(list1[idx1])
            idx1 += 1
    return union


def shuntingYardRPN(tokens):
    """
    mebuat reverse polish notations dari list of tokens
    dengan menggunakan shunting yard algorithm
    
    Params
    ----
    tokens: List[str]
        list of token.

    Returns
    -----
    List[str]
        hasil reverse polish notation/postfix notation dari shunting yard algorithm
    """
    precedence = {}
    precedence['NOT'] = 3
    precedence['AND'] = 2
    precedence['OR'] = 1
    precedence['('] = 0
    precedence[')'] = 0 
    
    output = []
    stack = []

    for token in tokens:
        if (token == '('):
            stack.append(token)

        elif (token == ')'):
            operator = stack.pop()
            while operator != '(':
                output.append(operator)
                operator = stack.pop()

        elif (token in precedence):
            if (stack):
                operator = stack[-1]
                while (stack and precedence[token] < precedence[operator]):
                    output.append(operator)
                    stack.pop()
                    if (stack):
                        operator = stack[-1]
            
            stack.append(token)
        else:
            # term 
            output.append(token.lower())
    
    while (stack):
        output.append(stack.pop())
    return output


def not_operation(list1, indexed_docIDs):
    """
    operasi negasi untuk list1
    
    Params
    ----
    list1: List[int]
        posting lists ke-1
    indexed_docIDs: List[int]
        semua docIDs yang sudah di index oleh search engine

    Returns
    -----
    List[Int]
        hasil operasi negasi
    """
    if (not list1):
        return indexed_docIDs

    result = []
    list1Set = set(list1)
    for item in indexed_docIDs:
        if item not in list1Set:
            result.append(item)
    return result
    

In [21]:
import re
from abc import ABC, abstractmethod



class BooleanOperator(ABC):
    @abstractmethod
    def evaluate(self, mapper):
        pass

class Term(BooleanOperator):
    def __init__(self, term, term_id_map):
        self.term = term
        self.term_id_map = term_id_map

    def evaluate(self, mapper):
        # return set(index.get(self.term, []))
        token = stemmer.stem(self.term)
        term_id = self.term_id_map.str_to_id.get(token.lower())
        posting_list = list(set(mapper[term_id]))
        return set(posting_list)

class And(BooleanOperator):
    def __init__(self, left, right, term_id_map):
        self.left = left
        self.right = right
        self.term_id_map = term_id_map

    def evaluate(self, mapper):
        left_result = self.left.evaluate(mapper)
        right_result = self.right.evaluate(mapper)
        return left_result.intersection(right_result)

class Or(BooleanOperator):
    def __init__(self, left, right, term_id_map):
        self.left = left
        self.right = right
        self.term_id_map = term_id_map

    def evaluate(self, mapper):
        left_result = self.left.evaluate(mapper)
        right_result = self.right.evaluate(mapper)
        return left_result.union(right_result)

class Not(BooleanOperator):
    def __init__(self, operand, term_id_map, indexed_docIDs):
        self.operand = operand
        self.term_id_map = term_id_map
        self.indexed_docIDs = indexed_docIDs

    def evaluate(self, mapper):
        operand_result = self.operand.evaluate(mapper)
        # all_docs = set().union(*index.values())
        not_result = not_operation(operand_result, self.indexed_docIDs )
        
        return not_result



In [22]:
import collections
import re

non_optimized_running_time = []
optimized_running_time = []

class SIPMI_BSBIIndexer(SIPMI_BSBIIndexer):
  def getPostingList(self, token, mapper):
     token = stemmer.stem(token)
     term_id = self.term_id_map.str_to_id.get(token.lower())
     posting_list = mapper[term_id]
      
     return posting_list
  
    
  def query(self, query):
      """
      boolean query (support operasi AND, OR, dan Negation).
      
      Params
      ------
      query: str
          query untuk search engine

      Returns
      ------
      List[str]
          list of title documents yang relevan dengan query
      """
      start = time.time()
      if len(self.term_id_map) == 0 or len(self.doc_id_map) == 0:
          self.load()
     
      with InvertedIndex(self.index_name, directory=self.output_dir
                                ) as mapper:
          result = None
          
          result = list(set(self.processQuery(query, mapper)))
          end = time.time()
          print(f"total waktu yang dibutuhkan untuk melakukan query (non-optimized): {(end-start)*10**3:.03f}ms")
          non_optimized_running_time.append(f"{(end-start)*10**3:.03f}ms")
          return [self.doc_id_map[r] for r in result]
          

  def processQuery(self, query, mapper):
        """"
        memproses query dengan cara membuat reverse polish notations dari string query, lalu
        mengevaluasi reverse polish notations sesuai dengan tipe operator nya (AND/OR/NOT).

        Params
        -------
        query: str
            string query ke search engine
        mapper: InvertedIndex
            mapper untuk memetakan termID ke postingListsnya

        Returns
        ------
        List[int]
            list of docID yang relevan dengan query
        """
        
        query = query.replace('(', '( ')
        query = query.replace(')', ' )')
        query  = query.split(' ')

        indexed_docIDs = list(range(1, self.docLen +1))
                              
      
        stack = []
        rpn_queue = collections.deque(shuntingYardRPN(query))

        while rpn_queue:
            token = rpn_queue.popleft()
            if token not in ('AND', 'NOT', 'OR'):
                # term
                token = stemmer.stem(token)
                term_id = self.term_id_map.str_to_id.get(token.lower())
                posting_list = mapper[term_id]
                
                
                stack.append(posting_list)
            else:
                if (token == 'AND'):
                    right = stack.pop()
                    left = stack.pop()
                    stack.append(sorted_intersect(left, right))
                elif (token == 'OR'):
                    right = stack.pop()
                    left = stack.pop()
                    stack.append(sorted_union(left, right))
                elif (token == 'NOT'):
                    right = stack.pop()
                    stack.append(not_operation(right, indexed_docIDs))
        if len(stack) != 1:
            print("error, harusnya panjang stacknya tinggal 1")
            return None
        return stack.pop()


  def parse_query(self, query, mapper):
    tokens = re.findall(r'\(|\)|AND|OR|NOT|\w+', query.upper())
    
    def parse_expression():
        stack = []
        output = []
        precedence = {'OR': 1, 'AND': 2, 'NOT': 3}
        
        for token in tokens:
            if token == '(':
                stack.append(token)
            elif token == ')':
                while stack and stack[-1] != '(':
                    output.append(stack.pop())
                stack.pop()  # Remove '('
            elif token in precedence:
                while stack and stack[-1] != '(' and precedence.get(stack[-1], 0) >= precedence[token]:
                    output.append(stack.pop())
                stack.append(token)
            else:
                output.append(Term(token.lower(), self.term_id_map))
        
        while stack:
            output.append(stack.pop())
        
        return output
    indexed_docIDs = list(range(1, self.docLen +1))

    def evaluate_postfix(postfix, mapper):
        stack = []
        for token in postfix:
            if isinstance(token, Term):
                stack.append(token)
            elif token == 'AND':
                right, left = stack.pop(), stack.pop()
                stack.append(And(left, right, self.term_id_map))
            elif token == 'OR':
                right, left = stack.pop(), stack.pop()
                stack.append(Or(left, right, self.term_id_map))
            elif token == 'NOT':
                operand = stack.pop()
                stack.append(Not(operand, self.term_id_map, indexed_docIDs))
        return stack.pop()
    
    postfix = parse_expression()
    return evaluate_postfix(postfix, mapper)

                
  def optimize_query(self, query_tree, mapper):
    indexed_docIDs = list(range(1, self.docLen +1))
    if isinstance(query_tree, Term):
        return query_tree
    elif isinstance(query_tree, Not):
        return Not(self.optimize_query(query_tree.operand, mapper), self.term_id_map, indexed_docIDs)
    elif isinstance(query_tree, (And, Or)):
        left = self.optimize_query(query_tree.left, mapper)
        right = self.optimize_query(query_tree.right, mapper)
        if isinstance(left, Term) and isinstance(right, Term):
          
            
            left_freq = len(self.getPostingList(left.term, mapper))
            right_freq = len(self.getPostingList(right.term, mapper))
            if left_freq > right_freq:
                left, right = right, left
        if isinstance(query_tree, And):
            return And(left, right, self.term_id_map)
        else:
            return Or(left, right, self.term_id_map)
    return query_tree

  def optimizedQuery(self, query):
    start = time.time()
    if len(self.term_id_map) == 0 or len(self.doc_id_map) == 0:
        self.load()
    with InvertedIndex(self.index_name, directory=self.output_dir ) as mapper:
        
        query_tree = self.parse_query(query, mapper)
        optimized_tree = self.optimize_query(query_tree, mapper)
        resultSet =  optimized_tree.evaluate(mapper)       
        end = time.time()
        print(f"total waktu yang dibutuhkan untuk melakukan query (optimized): {(end-start)*10**3:.03f}ms")
        optimized_running_time.append(f"{(end-start)*10**3:.03f}ms")
        return [self.doc_id_map[r] for r in list(resultSet)]
        

In [23]:
import math


class SIPMI_BSBIIndexer(SIPMI_BSBIIndexer):
    def build_idf(self ):
        try:
            with open("./output_dir/docs.dict", 'rb') as f:
                docs = pkl.load(f)
            self.total_doc_num = len(docs)
            self.idf = {}
            
            metadata_file_path = os.path.join("output_dir", "SIPMI_BSBI_lintang"+'.dict')
            with open(metadata_file_path, 'rb') as f:
                postings_dict, terms = pkl.load(f)

            index_file_path = os.path.join("output_dir", "SIPMI_BSBI_lintang"+'.index')
            index_file = open(index_file_path, 'rb+')
            for t_id, (start_position, df, length_in_bytes) in postings_dict.items():
                index_file.seek(start_position)
                postings_list = decode_postings_list(index_file.read(length_in_bytes))
                
                
                df = len((postings_list))
                idf = math.log10(self.total_doc_num) - math.log10(df+1)
                self.idf[t_id] = idf
            index_file.close()
                
        except FileNotFoundError:
            print("index file not found!")

    def get_idf(self, term):
        smother =  math.log10(self.total_doc_num) - math.log10(1)
        return self.idf.get(term, smother)


    def compute_tf_idf(self, query: str):
      
        
        with InvertedIndex(self.index_name, directory=self.output_dir
                    ) as mapper:
            
            query = stemmer.stem(query)
            terms = nltk.word_tokenize(query)
            w_t_d = {}
            for term in terms:
               
                term_id = self.term_id_map.str_to_id.get(term.lower())
                postings_list = mapper[term_id]
               
                tf = {}
                for docID in postings_list:
                    if docID not in tf:
                        tf[docID] = 1 / self.docWordCount[docID]
                    else:
                        tf[docID] += 1 / self.docWordCount[docID]
                
                curr_idf = 0
                if term_id in self.idf:
                    curr_idf = self.idf[term_id]
                
                for docID in tf.keys():
                    w_t_d[docID, term_id] =  (1 + math.log10(tf[docID])) * curr_idf
     
            documentScores = {}
            
            for (docID, term_id), tfidf  in w_t_d.items():
                if docID not in documentScores:
                    documentScores[docID] = tfidf
                else:
                    documentScores[docID] += tfidf
        
            sortedDocumentScores = dict(sorted(documentScores.items(), key=lambda item: item[1], reverse=True))
        doc_title_map = {}
        for docID in documentScores.keys():
            doc_title_map[docID] = self.doc_id_map[docID]
        sortedRes = {}
        for k, v in sortedDocumentScores.items():
            if v != 0 and len(sortedRes) <= 10:
                sortedRes[k] = v
        
        res = list(map(lambda x: (x, doc_title_map[x]) ,sortedRes.keys()))
        return res

In [24]:
SIPMI_BSBI_instance = SIPMI_BSBIIndexer(file_path= "./News.csv", output_dir = 'output_dir', )
SIPMI_BSBI_instance.index()
SIPMI_BSBI_instance.build_idf()

berhasil memproses index ke- 0
berhasil memproses index ke- 1
berhasil memproses index ke- 2
berhasil memproses index ke- 3
berhasil memproses index ke- 4
berhasil memproses index ke- 5
berhasil memproses index ke- 6
berhasil memproses index ke- 7
berhasil memproses index ke- 8
berhasil memproses index ke- 9
berhasil memproses index ke- 10
total waktu yang dibutuhkan untuk indexing seluruh dokumen: 25529.257ms


### 5 Boolean Query non-optimized & Optimized



In [25]:
notOptimized = SIPMI_BSBI_instance.query('Ganjar AND PDIP AND Megawati AND NOT Jokowi')
notOptimized

total waktu yang dibutuhkan untuk melakukan query (non-optimized): 18.221ms


['Survei LP3ES: PDIP Teratas, Demokrat Salip Dominasi Gerindra dan Golkar',
 'Usai jadi Host Youtube, Sepak Terjang Ganjar Pranowo Dianggap Sudah Kelewatan',
 'Politikus PDIP: Pertemuan Mega, Puan, Hasto dan Pacul Bahas Hal Super Spesifik',
 'Wasekjen PDIP Mengaku Belum Dengar Soal Teguran Megawati ke Bambang Pacul',
 'Utut Adianto: Saya Sarankan Puan Maharani Konsentrasi sebagai Ketua DPR',
 'DPP PDIP Tegaskan Tidak Ada Keputusan Persilakan Ganjar Pranowo Cari Parpol Lain',
 'Saran Saiful Mujani Buat Ganjar Pranowo: Masuk Timses Puan Maharani',
 'Membaca Peluang Koalisi Nasionalis-Religius Usung Capres di 2024',
 'PDIP: Megawati Tidak Mengurusi Ribut-ribut Ganjar Pranowo dan Puan Maharani',
 'Survei SMRC: Elektabilitas Prabowo-Ganjar Naik, Anies di Posisi Ketiga',
 'Survei: Bukan Prabowo, Milenial dan Gen Z Unggulkan Ganjar-Ridwan Kamil',
 'Cerita Puan Maharani Dimarahi Megawati']

In [26]:
optimized = SIPMI_BSBI_instance.optimizedQuery('Ganjar AND PDIP AND Megawati AND NOT Jokowi')
optimized

total waktu yang dibutuhkan untuk melakukan query (optimized): 21.855ms


['Survei LP3ES: PDIP Teratas, Demokrat Salip Dominasi Gerindra dan Golkar',
 'Usai jadi Host Youtube, Sepak Terjang Ganjar Pranowo Dianggap Sudah Kelewatan',
 'Politikus PDIP: Pertemuan Mega, Puan, Hasto dan Pacul Bahas Hal Super Spesifik',
 'Wasekjen PDIP Mengaku Belum Dengar Soal Teguran Megawati ke Bambang Pacul',
 'Utut Adianto: Saya Sarankan Puan Maharani Konsentrasi sebagai Ketua DPR',
 'DPP PDIP Tegaskan Tidak Ada Keputusan Persilakan Ganjar Pranowo Cari Parpol Lain',
 'Saran Saiful Mujani Buat Ganjar Pranowo: Masuk Timses Puan Maharani',
 'Membaca Peluang Koalisi Nasionalis-Religius Usung Capres di 2024',
 'PDIP: Megawati Tidak Mengurusi Ribut-ribut Ganjar Pranowo dan Puan Maharani',
 'Survei SMRC: Elektabilitas Prabowo-Ganjar Naik, Anies di Posisi Ketiga',
 'Survei: Bukan Prabowo, Milenial dan Gen Z Unggulkan Ganjar-Ridwan Kamil',
 'Cerita Puan Maharani Dimarahi Megawati']

In [27]:
notOptimized = SIPMI_BSBI_instance.query('Joget AND Tiktok OR (Viral AND 2020)')
notOptimized

total waktu yang dibutuhkan untuk melakukan query (non-optimized): 14.894ms


['Usai Dinaiki Lucinta Luna, Lumba-Lumba Hidung Botol Akhirnya Dievakuasi',
 'Viral Video Camat di Cilacap Dapat Pesawat, Sekda Sebut Cuma Bercanda',
 'Viral Foto Jenazah Laskar FPI Tersenyum, Ternyata Orangnya Masih Hidup',
 'Cek Fakta Video Hoaks Nan Manyabuik Jadwal Varian Covid-19 Diluncurkan',
 '9 Tahun Beraksi Embat Ratusan Motor, Sindikat Curanmor Dibekuk Polisi',
 'Cewek Ini Pernah Diselingkuhi & Dikasari,Kini Nasibnya Bak Ratu Usai Punya Cowok Baru',
 'Simak Hoaks Seputar Covid-19 yang Beredar Lewat WhatsApp dalam Sepekan',
 'Video Gadis Palestina Berani Lawan Tentara Israel, Tak Ada Takut-takutnya',
 'Viral Video Polisi Diduga Terima Pungli di Tol, Kapolri Sebut Kejadian Tahun Lalu',
 'Viral Foto Bupati Solok Masuk Masjid Tanpa Lepas Sepatu',
 'Perempuan Ini Mencuri Tiket Final Euro 2020 Pacarnya yang Selingkuh',
 'Rahasia Sukses Bangun Usaha Ala Pemiliki Ayam Keprabon Express',
 'Cek Fakta: Tidak Benar dalam Video Ini WNA Masuk Indonesia Lewat Bandara Soetta saat PPKM',
 'Ce

In [28]:
optimized = SIPMI_BSBI_instance.optimizedQuery('Joget AND Tiktok OR (Viral AND 2020)')
optimized

total waktu yang dibutuhkan untuk melakukan query (optimized): 17.276ms


['Usai Dinaiki Lucinta Luna, Lumba-Lumba Hidung Botol Akhirnya Dievakuasi',
 'Viral Video Camat di Cilacap Dapat Pesawat, Sekda Sebut Cuma Bercanda',
 'Viral Foto Jenazah Laskar FPI Tersenyum, Ternyata Orangnya Masih Hidup',
 'Cek Fakta Video Hoaks Nan Manyabuik Jadwal Varian Covid-19 Diluncurkan',
 '9 Tahun Beraksi Embat Ratusan Motor, Sindikat Curanmor Dibekuk Polisi',
 'Cewek Ini Pernah Diselingkuhi & Dikasari,Kini Nasibnya Bak Ratu Usai Punya Cowok Baru',
 'Video Gadis Palestina Berani Lawan Tentara Israel, Tak Ada Takut-takutnya',
 'Simak Hoaks Seputar Covid-19 yang Beredar Lewat WhatsApp dalam Sepekan',
 'Viral Video Polisi Diduga Terima Pungli di Tol, Kapolri Sebut Kejadian Tahun Lalu',
 'Viral Foto Bupati Solok Masuk Masjid Tanpa Lepas Sepatu',
 'Perempuan Ini Mencuri Tiket Final Euro 2020 Pacarnya yang Selingkuh',
 'Rahasia Sukses Bangun Usaha Ala Pemiliki Ayam Keprabon Express',
 'Cek Fakta: Tidak Benar dalam Video Ini WNA Masuk Indonesia Lewat Bandara Soetta saat PPKM',
 'Ce

In [29]:
notOptimized = SIPMI_BSBI_instance.query('Jokowi AND NOT Mahasiswa AND UGM OR (Gibran OR Bodoh)')
notOptimized

total waktu yang dibutuhkan untuk melakukan query (non-optimized): 16.311ms


['Lama Jadi Masalah Besar, Lini Serang MU Akhirnya Bisa Diperbaiki Berkat 2 Pemain Ini',
 'IDI: Rumah Sakit Memang Penuh, Keadaan Darurat Ini Nyata',
 'Ucapan Idul Adha dari BEI hingga Perusahaan Sekuritas Sampaikan Pesan Penting',
 'Wanita Cantik Minta Selfie Sama Gibran, Ternyata Anak Buah Pejabat Tinggi di RI',
 '6 Nasihat Megawati untuk Anak-Anak Indonesia di Hari Anak Nasional 2021',
 'Polisi Militer Saksikan Gibran Rakabuming Semprotkan Desinfektan ke Ajudan',
 '48 Kata-Kata Masa Bodoh, Bisa Jadi Sindiran Sekaligus Bentuk Rasa Tidak Peduli',
 '31 Ucapan Terima Kasih untuk Teman Dekat, Bukan Hanya Sekedar Pujian',
 '5 Pengakuan Gibran Rakabuming Usai Dinyatakan Positif Covid-19',
 '46 Ucapan Terima Kasih kepada Guru Ketika Momen Perpisahan, Penuh Makna',
 'Berstatus Anak Presiden, Begini Gaya Gibran Sambut Panglima TNI dan Kasad Andika',
 'Pengendara yang Dehidrasi Sama Bahayanya dengan Pengemudi Mabuk',
 'Potret Lawas Annisa Pohan Jelang Jadi Pengantin, Alisnya Ramai Disorot',
 "

In [30]:
optimized = SIPMI_BSBI_instance.optimizedQuery('Jokowi AND NOT Mahasiswa AND UGM OR (Gibran OR Bodoh)')
optimized

total waktu yang dibutuhkan untuk melakukan query (optimized): 21.806ms


['Lama Jadi Masalah Besar, Lini Serang MU Akhirnya Bisa Diperbaiki Berkat 2 Pemain Ini',
 'IDI: Rumah Sakit Memang Penuh, Keadaan Darurat Ini Nyata',
 'Ucapan Idul Adha dari BEI hingga Perusahaan Sekuritas Sampaikan Pesan Penting',
 'Wanita Cantik Minta Selfie Sama Gibran, Ternyata Anak Buah Pejabat Tinggi di RI',
 'Polisi Militer Saksikan Gibran Rakabuming Semprotkan Desinfektan ke Ajudan',
 '6 Nasihat Megawati untuk Anak-Anak Indonesia di Hari Anak Nasional 2021',
 '48 Kata-Kata Masa Bodoh, Bisa Jadi Sindiran Sekaligus Bentuk Rasa Tidak Peduli',
 '31 Ucapan Terima Kasih untuk Teman Dekat, Bukan Hanya Sekedar Pujian',
 '5 Pengakuan Gibran Rakabuming Usai Dinyatakan Positif Covid-19',
 '46 Ucapan Terima Kasih kepada Guru Ketika Momen Perpisahan, Penuh Makna',
 'Berstatus Anak Presiden, Begini Gaya Gibran Sambut Panglima TNI dan Kasad Andika',
 'Pengendara yang Dehidrasi Sama Bahayanya dengan Pengemudi Mabuk',
 '50 Kata-Kata Motivasi Keren yang Bisa Bikin Hidup Tetap Optimis',
 "Ma'ruf 

In [31]:
notOptimized = SIPMI_BSBI_instance.query('Lagu AND (Dangdut OR Koplo) AND NOT (Jazz OR Pop)')
notOptimized

total waktu yang dibutuhkan untuk melakukan query (non-optimized): 17.458ms


['Ubah Gaya dan Warna Rambut, Ini 6 Potret Terbaru Rizki DA Makin Macho',
 '6 Potret Maternity Shoot Nella Kharisma, Romantis Ditemani Dory Harsa',
 'KPI Harap Radio Tak Lagi Putar Lagu Bermuatan Pornografi',
 "Cerita Atlet Wushu yang Sakit Kanker Bisa Survive Karena Lagu 'Syahdu' Rhoma Irama",
 '6 Potret Chevra Yolandi, Vokalis Band Papinka yang Dikabarkan Kekasih Via Vallen',
 "Aurel Dewanda Rilis Single Perdana 'Salah Paham'"]

In [32]:
optmized = SIPMI_BSBI_instance.optimizedQuery('Lagu AND (Dangdut OR Koplo) AND NOT (Jazz OR Pop)')
optmized

total waktu yang dibutuhkan untuk melakukan query (optimized): 17.992ms


['Ubah Gaya dan Warna Rambut, Ini 6 Potret Terbaru Rizki DA Makin Macho',
 '6 Potret Maternity Shoot Nella Kharisma, Romantis Ditemani Dory Harsa',
 'KPI Harap Radio Tak Lagi Putar Lagu Bermuatan Pornografi',
 "Cerita Atlet Wushu yang Sakit Kanker Bisa Survive Karena Lagu 'Syahdu' Rhoma Irama",
 '6 Potret Chevra Yolandi, Vokalis Band Papinka yang Dikabarkan Kekasih Via Vallen',
 "Aurel Dewanda Rilis Single Perdana 'Salah Paham'"]

In [33]:
notOptimized = SIPMI_BSBI_instance.optimizedQuery('Olimpiade AND Tokyo AND NOT (Prancis OR jerman) OR Badminton')
notOptimized

total waktu yang dibutuhkan untuk melakukan query (optimized): 18.472ms


['Infografis Jadwal Bulu Tangkis Indonesia di Olimpiade Tokyo 2020',
 'Joe Biden Puji Keberhasilan Jepang Selenggarakan Olimpiade Tokyo 2020',
 'Greysia Polii dan Apriyani Rahayu Raih Medali Emas di Olimpiade Tokyo 2020',
 'Hasil Menembak Olimpiade Tokyo 2020 : Vidya Rafika Gagal Buka Peluang Raih Medali',
 'Jadwal Siaran Langsung Atlet Indonesia di Olimpiade Tokyo 2020, Sabtu 24 Juli 2021',
 'Link Live Streaming Bulu Tangkis Indonesia Olimpiade Tokyo 2020 di Vidio : Misi Dulang Medali Dimulai',
 'Jadwal Olimpiade Tokyo 2020 : Tim Inggris Raya Ingin Tampil Tanpa Emosi di Laga Pembuka',
 'Waduh, Ahli Kesehatan Serukan Penundaan Olimpiade Tokyo 2020 Usai Belasan Atlet Positif Covid-19',
 'TV Korsel Minta Maaf karena Pakai Gambar Pizza Saat Tayangkan Tim Olimpiade Italia',
 'Update dan Jadwal Laga Bulu Tangkis Kontingen Indonesia Olimpiade Tokyo 2020',
 'Jadwal Bulu Tangkis Olimpiade 2020 : Andalan Indonesia Dulang Medali',
 'Usain Bolt Khawatirkan Penampilan Atlet Jamaika di Olimpiade To

In [34]:
optimized = SIPMI_BSBI_instance.optimizedQuery('Olimpiade AND Tokyo AND NOT (Prancis OR jerman) OR Badminton')
optimized

total waktu yang dibutuhkan untuk melakukan query (optimized): 19.670ms


['Infografis Jadwal Bulu Tangkis Indonesia di Olimpiade Tokyo 2020',
 'Joe Biden Puji Keberhasilan Jepang Selenggarakan Olimpiade Tokyo 2020',
 'Greysia Polii dan Apriyani Rahayu Raih Medali Emas di Olimpiade Tokyo 2020',
 'Hasil Menembak Olimpiade Tokyo 2020 : Vidya Rafika Gagal Buka Peluang Raih Medali',
 'Jadwal Siaran Langsung Atlet Indonesia di Olimpiade Tokyo 2020, Sabtu 24 Juli 2021',
 'Link Live Streaming Bulu Tangkis Indonesia Olimpiade Tokyo 2020 di Vidio : Misi Dulang Medali Dimulai',
 'Jadwal Olimpiade Tokyo 2020 : Tim Inggris Raya Ingin Tampil Tanpa Emosi di Laga Pembuka',
 'Waduh, Ahli Kesehatan Serukan Penundaan Olimpiade Tokyo 2020 Usai Belasan Atlet Positif Covid-19',
 'TV Korsel Minta Maaf karena Pakai Gambar Pizza Saat Tayangkan Tim Olimpiade Italia',
 'Update dan Jadwal Laga Bulu Tangkis Kontingen Indonesia Olimpiade Tokyo 2020',
 'Jadwal Bulu Tangkis Olimpiade 2020 : Andalan Indonesia Dulang Medali',
 'Usain Bolt Khawatirkan Penampilan Atlet Jamaika di Olimpiade To

In [35]:
notOptimized = SIPMI_BSBI_instance.query('(Saham AND Bank) OR (Saham AND Energi) AND NOT (Teknologi AND Kesehatan)')
notOptimized

total waktu yang dibutuhkan untuk melakukan query (non-optimized): 22.778ms


['BTN dan Indofarma Perkuat Kerjasama Layanan Perbankan',
 'Anak Usaha Telkom dan Pertamina Bakal Melantai di Bursa 2021',
 'BRI Kejar Porsi Kredit Mikro Hingga 45% di 2025',
 'Mayoritas Saham Bukalapak Masih Dimiliki Investor Dalam Negeri',
 'Naik 19 Kali Lipat, Investor Saham Syariah Tembus 93.870 di 2021',
 'Top 3: Restrukturisasi Utang Waskita Karya Beri Ruang Perbaiki Kinerja',
 'BEI Gembok Sementara Perdagangan Saham BINA',
 'Perdana Gapuraprima Ungkap Keuntungan Investasi Properti saat Pandemi COVID-19',
 'Ada PPKM, Kinerja Emiten Properti Terganggu',
 'Perdana Gapuraprima Siapkan Belanja Modal Rp 225 Miliar, untuk Apa Saja?',
 'Induk Usaha HM Sampoerna Akuisisi Fertin Pharma Rp 12 Triliun',
 'Top 3: Rektor UI Ari Kuncoro Mundur sebagai Wakil Komisaris Utama BRI',
 'Buyung Poetra Sembada Tebar Dividen Rp 9,67 Miliar',
 'Aksi Beli Investor Asing Capai Rp 807 Miliar, Simak Saham Pilihan Ini',
 'Laba CIMB Niaga Naik 22 Persen Jadi Rp2,1 Triliun di Semester I-2021',
 'BI Tahan Bunga

In [36]:
optimized = SIPMI_BSBI_instance.query('(Saham AND Bank) OR (Saham AND Energi) AND NOT (Teknologi AND Kesehatan)')
optimized

total waktu yang dibutuhkan untuk melakukan query (non-optimized): 22.710ms


['BTN dan Indofarma Perkuat Kerjasama Layanan Perbankan',
 'Anak Usaha Telkom dan Pertamina Bakal Melantai di Bursa 2021',
 'BRI Kejar Porsi Kredit Mikro Hingga 45% di 2025',
 'Mayoritas Saham Bukalapak Masih Dimiliki Investor Dalam Negeri',
 'Naik 19 Kali Lipat, Investor Saham Syariah Tembus 93.870 di 2021',
 'Top 3: Restrukturisasi Utang Waskita Karya Beri Ruang Perbaiki Kinerja',
 'BEI Gembok Sementara Perdagangan Saham BINA',
 'Perdana Gapuraprima Ungkap Keuntungan Investasi Properti saat Pandemi COVID-19',
 'Ada PPKM, Kinerja Emiten Properti Terganggu',
 'Perdana Gapuraprima Siapkan Belanja Modal Rp 225 Miliar, untuk Apa Saja?',
 'Induk Usaha HM Sampoerna Akuisisi Fertin Pharma Rp 12 Triliun',
 'Top 3: Rektor UI Ari Kuncoro Mundur sebagai Wakil Komisaris Utama BRI',
 'Buyung Poetra Sembada Tebar Dividen Rp 9,67 Miliar',
 'Aksi Beli Investor Asing Capai Rp 807 Miliar, Simak Saham Pilihan Ini',
 'Laba CIMB Niaga Naik 22 Persen Jadi Rp2,1 Triliun di Semester I-2021',
 'BI Tahan Bunga

In [37]:
from tabulate import tabulate
print(non_optimized_running_time)
print(optimized_running_time)

running_time_table = [['Ganjar AND PDIP AND Megawati AND NOT Jokowi', non_optimized_running_time[0], optimized_running_time[0]],
         ['Joget AND Tiktok OR (Viral AND 2020)',  non_optimized_running_time[1], optimized_running_time[1]],
        ['Jokowi AND NOT Mahasiswa AND UGM OR (Gibran OR Bodoh)',  non_optimized_running_time[2], optimized_running_time[2]],
         ['Lagu AND (Dangdut OR Koplo) AND NOT (Jazz OR Pop)',  non_optimized_running_time[3], optimized_running_time[3]],
         ['Olimpiade AND Tokyo AND NOT (Prancis OR jerman) OR Badminton',  non_optimized_running_time[4], optimized_running_time[4]],
         ['(Saham AND Bank) OR (Saham AND Energi) AND NOT (Teknologi AND Kesehatan)',  non_optimized_running_time[5], optimized_running_time[5]]
        ]
column = ["Query", "Non-Optimized", "Optimized"]
print(tabulate(running_time_table, headers=column, tablefmt="grid"))



['18.221ms', '14.894ms', '16.311ms', '17.458ms', '22.778ms', '22.710ms']
['21.855ms', '17.276ms', '21.806ms', '17.992ms', '18.472ms', '19.670ms']
+--------------------------------------------------------------------------+-----------------+-------------+
| Query                                                                    | Non-Optimized   | Optimized   |
+==========================================================================+=================+=============+
| Ganjar AND PDIP AND Megawati AND NOT Jokowi                              | 18.221ms        | 21.855ms    |
+--------------------------------------------------------------------------+-----------------+-------------+
| Joget AND Tiktok OR (Viral AND 2020)                                     | 14.894ms        | 17.276ms    |
+--------------------------------------------------------------------------+-----------------+-------------+
| Jokowi AND NOT Mahasiswa AND UGM OR (Gibran OR Bodoh)                    | 16.311ms      

# Coba TF-IDF


In [38]:
ss = SIPMI_BSBI_instance.compute_tf_idf(query="Gibran Solo")
ss

[(10601,
  'Viral Video Lawas Momen Gibran Rakabuming yang Bikin Najwa Shihab Deg-Degan'),
 (10398,
  'Dibilang Lumpuh Total, Reaksi Gibran Rakabuming Santuy Banget Bikin Salut'),
 (2098, 'Piala Wali Kota Solo Ditunda Lagi, Persib: Keputusan Bijak'),
 (4795,
  'Kata Menhub saat Jajal KRL Yogyakarta-Solo: Semoga Bisa Diperpanjang Sampai Madiun'),
 (7988, 'Enam Jalan Protokol di Surakarta Dilakukan Penyekatan PPKM Darurat'),
 (2115, 'Persib Tak Bawa Seluruh Pemain Asing ke Piala Wali Kota Solo'),
 (11499,
  'Pasien Covid-19 Asal Kudus Isolasi di Asrama Haji Donohudan Boyolali Meninggal Dunia'),
 (11474,
  'Warga Kudus Meninggal Saat Isolasi di Donohudan Boyolali Bertambah Menjadi 4 Orang'),
 (8441, 'Temui PKL di Solo, Kemnaker Bagikan Bantuan TKM'),
 (12502, 'Ziarah ke Astana Oetara, Ketum Golkar Masih Keturunan Mangkunegara'),
 (5079, 'Polisi Cegat Pengiriman Ratusan Liter Miras dari Solo ke Kebumen')]

In [39]:
ss = SIPMI_BSBI_instance.compute_tf_idf(query="Covid PPKM menyebabkan work home dan sekolah online")
ss

[(9301, 'Konsumsi BBM dan Elpiji di Sumbar Menurun Selama Penerapan PPKM'),
 (4, 'Infografis Perbedaan Aturan PPKM Level 3 dan 4'),
 (2489, 'Resmi Diperpanjang, Ini 6 Curhatan Kocak Netizen Soal PPKM Level 4'),
 (13072, 'Samarinda Diputuskan PPKM Darurat 21-31 Juli 2021'),
 (1459,
  'PPKM Darurat Jatim Diperpanjang hingga 2 Agustus Hoaks, Ini yang Benar'),
 (11809,
  'Pemerintah Putuskan Takbiran dan Salat Iduladha di Wilayah PPKM Darurat Ditiadakan'),
 (8571,
  'Kemendagri Terbitkan Aturan Perpanjangan PPKM Mikro, Ada Pengetatan di 43 Daerah'),
 (4050,
  'Kemendagri Terbitkan Aturan Perpanjangan PPKM Mikro, Ada Pengetatan di 43 Daerah yang Diberlakukan Seperti PPKM Darurat'),
 (10704,
  'PP Muhammadiyah Nilai Keberhasilan PPKM Tergantung Konsistensi dan Disiplin'),
 (3346,
  'Cek Fakta: Tidak Benar PPKM Darurat Jatim Diperpanjang  hingga 2 Agustus 2021'),
 (6014,
  'Organisasi Pekerja Minta Pemerintah Salurkan Bantuan Subsidi Gaji Selama PPKM Darurat')]